In [212]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep
from collections import Counter

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [213]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\origen\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [214]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

df_cd_tr = pd.read_csv(generaPath('O', 'df_cd_tr.csv'))
df_cd_re = pd.read_csv(generaPath('O', 'df_cd_re.csv'))

df_cs_tr = pd.read_csv(generaPath('O', 'df_cs_tr.csv'))
df_cs_re = pd.read_csv(generaPath('O', 'df_cs_re.csv'))

df_cs_tr_sb = pd.read_csv(generaPath('O', 'df_cs_tr_sb.csv'))
df_cs_re_sb = pd.read_csv(generaPath('O', 'df_cs_re_sb.csv'))


df_sel_sb_tr = pd.read_csv(generaPath('O','df_sel_sb_tr.csv'))
df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_sb_re.csv'))

df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_si_tr.csv'))
df_sel_si_re = pd.read_csv(generaPath('O','df_sel_si_re.csv') )

sbs = pd.read_csv(generaPath('O','sbs.csv'))
singles = pd.read_csv(generaPath('O','singles.csv'))

df_tr_fu_filt = pd.read_csv(generaPath('O', 'df_tr_fu_filt_delta.csv'))
df_re_fu_filt = pd.read_csv(generaPath('O', 'df_re_fu_filt_delta.csv'))

df = pd.read_csv(generaPath('I', 'expdata2.csv'), parse_dates=['fecha'])

Filtra los numeros singles repetidos

In [215]:
def cantidad_no_rep():
    l= singles.sort_values("tupla")
    l2 = l.drop('R', 1)
    l3 = l2.drop_duplicates()
    print(l3)
    return l3

In [216]:
def validar_quinteto_list():
    l= singles.sort_values("tupla")
    l2 = l.drop('R', 1)
    l3 = l2.drop_duplicates()
    return l3

In [217]:
def secuencelist_unbuilder(secuence):
    num = str(secuence[0])
    partido = num.split(sep='|') if (num.find('|') >= 1) else num
    return partido

In [218]:
# def crea_sorteos(single,sb, secuence):
#     sorteo = []
#     enteros = []
#     # splitedtupla = unsplited_tupla(secuence)
#     #enteros = secuencelist_unbuilder(str(secuence[0]))
#     # print(splitedtupla)
#     print(" Tipo Enteros")
#     print(type(enteros))
#     print(enteros)
#     #sorteo = singles.iloc[enteros, 0]
#     #print(sorteo)
#     #sorteo.append(str(sb))
#     return sorteo

In [219]:
def secuencelist_builder(a,b,c,d,e):
    secuencia=[]
    secuencia.append(a)
    secuencia.append(b)
    secuencia.append(c) 
    secuencia.append(d)
    secuencia.append(e)
    palabra = '|'.join([str(et) for et in secuencia])
    return palabra, secuencia


In [220]:
def crea_secuencias(totalconteo):
    secuencias = []
    secuence = []
    lsecuence =[]
    secuencia = []
    i = 0
    a = i
    b = a + 1
    c = b + 1
    d = c + 1
    e = d + 1
    while (a <= totalconteo):
        while(b <= totalconteo):
            while(c <= totalconteo):
                while(d <= totalconteo):
                    while(e <= totalconteo):
                        secuencia, secuence = secuencelist_builder(a,b,c,d,e)
                        secuencias.append(secuencia)
                        lsecuence.append(secuence)
                        e += 1
                    d += 1
                    e = d + 1
                c += 1
                d = c + 1
                e = d + 1
            b += 1
            c = b + 1
            d = c + 1
            e = d + 1
        a = a + 1
        b = a + 1
        c = b + 1
        d = c + 1 
        e = d + 1
    return secuencias, lsecuence


Construye los numeros candidatos de acuerdo a los numeros que salieron en singles y sbs, itera para cada superbalota no esta usando el contenido de las superbalotas

In [221]:
def combinations_generator(sbs, l):
    sorteos = []
    lsecuence = []  
    secuencias = []
    cant = len(l) - 1
    secuencias, lsecuence = crea_secuencias(cant)
    cantsecuence = len(lsecuence)
    i = 0
    for rows in secuencias:
        lsec = lsecuence[int(i)]
        sorteo = l.iloc[lsec, 0]
        i = i + 1
        sorteos.append(sorteo.to_list())
    dft = sorteos
    # print("tipo de dft: "+ str(type(dft)))
    return dft

In [222]:
def combinations_getter_saver():
    secus = []
    l3 = cantidad_no_rep()
    # print("valor de l3")
    # print(len(l3))
    # print(singles)
    # print(sbs)
    dft_sorteos = combinations_generator(sbs, l3)
    dft = pd.DataFrame (dft_sorteos, columns = ['n1','n2','n3','n4','n5'])
    #print("tipo de dft: "+ str(type(dft)))
    dft.to_csv(generaPath('O','dft.csv'), header=['n1','n2','n3','n4','n5'], index=None, sep=',', mode='w' )
    #print(dft_sorteos)
    return dft

In [223]:
def construir_trio(n1,n2,n3):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3)
    return salida

In [224]:
def construir_list_trio(n1,n2,n3):
    salida = [n1,n2,n3]
    # salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    #print(salida)
    return salida

In [225]:
def construir_cuarteto(n1,n2,n3,n4):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4)
    return salida

In [226]:
def construir_list_cuarteto(n1,n2,n3,n4):
    salida = [n1,n2,n3,n4]
    return salida

In [227]:
def construir_list_cuarteto_sorteo(idSorteo, n1,n2,n3,n4):
    salida = [idSorteo, n1,n2,n3,n4]
    return salida

In [228]:
def construir_quinteto(n1,n2,n3,n4,n5):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    return salida

In [229]:
def construir_list_quinteto(n1,n2,n3,n4,n5):
    salida = [n1,n2,n3,n4,n5]
    # salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    # print(type(salida))
    return salida

In [230]:
def all_trios(n1,n2,n3,n4,n5):
    trio01 = construir_list_trio(n1,n2,n3)
    trio02 = construir_list_trio(n1,n2,n4)
    trio03 = construir_list_trio(n1,n2,n5)
    trio04 = construir_list_trio(n2,n3,n4)
    trio05 = construir_list_trio(n2,n3,n5)
    trio06 = construir_list_trio(n2,n4,n5)
    trio07 = construir_list_trio(n3,n4,n5)
    trio08 = construir_list_trio(n1,n3,n4)
    trio09 = construir_list_trio(n1,n3,n5)
    trio10 = construir_list_trio(n1,n4,n5)
    return trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [231]:
def all_cuartetos(n1,n2,n3,n4,n5):
    cuarteto1 = construir_list_cuarteto(n1, n2, n3, n4)
    cuarteto2 = construir_list_cuarteto(n1, n2, n3, n5)
    cuarteto3 = construir_list_cuarteto(n1, n2, n4, n5)
    cuarteto4 = construir_list_cuarteto(n1, n3, n4, n5)
    cuarteto5 = construir_list_cuarteto(n2, n3, n4, n5)
    return cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [232]:
def all_cuartetos_sorteos(idSorteo, n1,n2,n3,n4,n5):
    cuarteto1 = construir_list_cuarteto_sorteo(idSorteo, n1, n2, n3, n4)
    cuarteto2 = construir_list_cuarteto_sorteo(idSorteo, n1, n2, n3, n5)
    cuarteto3 = construir_list_cuarteto_sorteo(idSorteo, n1, n2, n4, n5)
    cuarteto4 = construir_list_cuarteto_sorteo(idSorteo, n1, n3, n4, n5)
    cuarteto5 = construir_list_cuarteto_sorteo(idSorteo, n2, n3, n4, n5)
    return cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [233]:
def find_previous_coincidence(n1,n2,n3,n4,n5, df):
    df_found = df.loc[(df.n1 == n1) & (df.n2 == n2) & (df.n3 == n3) & (df.n4 == n4) & (df.n5 == n5)]
    print(df_found)
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return str(df_found) , encontrados
    else:
        return str(df_found),  encontrados

In [234]:
def find_previous_coincidence_c(n1,n2,n3,n4, df):
    df_found = df.loc[(df.n1 == n1) & (df.n2 == n2) & (df.n3 == n3) & (df.n4 == n4)]
    print(df_found)
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return str(df_found) , encontrados
    else:
        return str(df_found),  encontrados

In [235]:
def find_previous_coincidence_t(n1,n2,n3, df):
    df_found = df.loc[(df.n1 == n1) & (df.n2 == n2) & (df.n3 == n3) ]  #& (df.n4 == n4) & (df.n5 == n5)
    print(df_found)
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return str(df_found) , encontrados
    else:
        return str(df_found),  encontrados

In [236]:
def validar_sumar_resultados_sorteos(n1,n2,n3,n4,n5, df_to_search):
    mensaje, suma = find_previous_coincidence(n1,n2,n3,n4,n5, df_to_search)
    # print(suma)
    return mensaje, suma

In [237]:
def validar_sumar_resultados_sorteos_c(n1,n2,n3,n4, df_to_search):
    mensaje, suma = find_previous_coincidence_c(n1,n2,n3,n4, df_to_search)
    # print(suma)
    return mensaje, suma

In [238]:
def validar_sumar_resultados_sorteos_t(n1,n2,n3,df_to_search):
    mensaje, suma = find_previous_coincidence_t(n1,n2,n3,df_to_search)
    # print(suma)
    return mensaje, suma

In [239]:
def ingresar_sorteo_validar_q(n1,n2,n3,n4,n5, quint):
    # quinteto = construir_quinteto(n1, n2, n3, n4, n5)
    quinteto = construir_list_quinteto(n1, n2, n3, n4, n5)
    quint.append(quinteto)
    return quinteto


In [240]:
def ingresar_sorteo_validar_c(n1,n2,n3,n4,n5, cuart):
    c1, c2, c3, c4, c5 = all_cuartetos(n1, n2, n3, n4, n5)
    cuart.append(c1)
    cuart.append(c2)
    cuart.append(c3)
    cuart.append(c4)
    cuart.append(c5)
    return cuart

In [241]:
def ingresar_sorteo_validar_c_sorteos(idSorteo, n1,n2,n3,n4,n5):
    cuart_sorteos = []
    c1, c2, c3, c4, c5 = all_cuartetos_sorteos(idSorteo, n1, n2, n3, n4, n5)
    cuart_sorteos.append(c1)
    cuart_sorteos.append(c2)
    cuart_sorteos.append(c3)
    cuart_sorteos.append(c4)
    cuart_sorteos.append(c5)
    return cuart_sorteos

In [242]:
def ingresar_sorteo_validar_t(n1,n2,n3,n4,n5, trios):
    t01, t02, t03, t04, t05, t06, t07, t08, t09, t10 = all_trios(n1, n2, n3, n4, n5)
    trios.append(t01)
    trios.append(t02)
    trios.append(t03)
    trios.append(t04)
    trios.append(t05)
    trios.append(t06)
    trios.append(t07)
    trios.append(t08)
    trios.append(t09)
    trios.append(t10)
    return trios

In [243]:
def sumar_tuplas(n1,n2,n3,n4,n5,df_exc):   
    rechazados = df_exc.loc[(df_exc.n1 == n1) & (df_exc.n2 == n2) & (df_exc.n3 == n3) & (df_exc.n4 == n4) & (df_exc.n5 == n5)]
    elegidos = df_exc.loc[(df_exc.n1 != n1) | (df_exc.n2 != n2) | (df_exc.n3 != n3) | (df_exc.n4 != n4) | (df_exc.n5 != n5  )]
    cant = len(rechazados)
    cant2 = len(elegidos)
    print("Cant Rechazados quintetos: " + str(cant) + " Cant Elegidos quitetos:" + str(cant2))
    return elegidos, rechazados, cant

In [244]:
def sumar_tuplas_c(n1,n2,n3,n4,df_exc):   
    rechazados = df_exc.loc[(df_exc.n1 == n1) & (df_exc.n2 == n2) & (df_exc.n3 == n3) & (df_exc.n4 == n4) ] #& (df_exc.n5 == n5)
    elegidos = df_exc.loc[(df_exc.n1 != n1) | (df_exc.n2 != n2) | (df_exc.n3 != n3) | (df_exc.n4 != n4) ] # | (df_exc.n5 != n5  )
    cant = len(rechazados)
    cant2 = len(elegidos)
    print("Cant Rechazados cuartetos: " + str(cant) + " Cant Elegidos cuartetos:" + str(cant2))
    return elegidos, rechazados, cant

In [245]:
def sumar_tuplas_t(n1,n2,n3,df_exc):   
    rechazados = df_exc.loc[(df_exc.n1 == n1) & (df_exc.n2 == n2) & (df_exc.n3 == n3) ] #& (df_exc.n4 == n4) & (df_exc.n5 == n5)
    elegidos = df_exc.loc[(df_exc.n1 != n1) | (df_exc.n2 != n2) | (df_exc.n3 != n3) ] #| (df_exc.n4 != n4) | (df_exc.n5 != n5  )
    cant = len(rechazados)
    cant2 = len(elegidos)
    print("Cant Rechazados trios: " + str(cant) + " Cant Elegidos trios:" + str(cant2))
    return elegidos, rechazados, cant

In [246]:
def couples_builder(dft):
    i = 0
    dictionary = {}
    can_elegidos = len(dft)
    print(can_elegidos)
    validos=[]
    quint=[]
    cuart=[]
    trios=[]
    for index, row in dft.iterrows():
        sbs = (row)
        n1 = int(sbs[0])
        n2 = int(sbs[1])
        n3 = int(sbs[2])
        n4 = int(sbs[3])
        n5 = int(sbs[4]) 
        porcentaje = ((i+1)/can_elegidos)*100
        totaltrios = 0
        totalcuartetos = 0
        totalquintetos = 0
        quint = ingresar_sorteo_validar_q(n1, n2, n3, n4, n5, quint)
        cuart = ingresar_sorteo_validar_c(n1, n2, n3, n4, n5, cuart)
        trios = ingresar_sorteo_validar_t(n1, n2, n3, n4, n5, trios)
        totales = 0
        totaltrios = 0
        i = i + 1
    print(len(quint))
    print(len(cuart))
    print(len(trios))
    return quint, cuart, trios

In [247]:
def couples_builder_sorteos(dft):
    i = 0
    cuart=[]
    for index, row in dft.iterrows():
        sbs = (row)
        idSorteo = int(sbs[0])
        n1 = int(sbs[1])
        n2 = int(sbs[2])
        n3 = int(sbs[3])
        n4 = int(sbs[4])
        n5 = int(sbs[5]) 
        cuart = ingresar_sorteo_validar_c_sorteos(idSorteo, n1, n2, n3, n4, n5)
        # trios = ingresar_sorteo_validar_t(n1, n2, n3, n4, n5, trios)
    print(len(cuart))
    return cuart

In [248]:
def validator2_m(df1, dft):
    # print("contenido de dft" + str(type(dft)))
    keys = list(dft.columns.values)
    # print(str(type(keys)) + " - " + str(keys))
    # print("contenido de df1" + str(type(df1)))
    keys2 = list(df1.columns.values)
    # print(str(type(keys2)) + " - " + str(keys2))
    i1 = df1.set_index(keys).index
    i2 = dft.set_index(keys).index
    df_res1 = df1[~i1.isin(i2)]
    df_res2 = df1[i1.isin(i2)]
    return df_res1, df_res2

In [249]:
dft = combinations_getter_saver()
df1 = df_tr_fu_filt
choosen, rejected = validator2_m(df1, dft)
print("Printing choosen ")
print(choosen)
print("Printing rejected")
print(rejected)


    tupla
12      1
9       6
15     11
4      12
5      13
1      17
8      24
6      25
14     27
7      30
13     31
3      34
2      36
0      42
Printing choosen 
     idSorteo  n1  n2  n3  n4  n5  sb  delta
0        2149   2   5  16  24  39  15  CKHÑF
1        2148   5  11  29  30  43  14  FQAME
2        2147   3  16  22  26  40  12  MFDNF
3        2146   1  15  20  31  35  15  NEKDI
4        2145   2   8  20  35  38   3  FLÑCG
..        ...  ..  ..  ..  ..  ..  ..    ...
474      1675   1  20  36  39  43  10  ROCDA
475      1674   5   6   7  12  37   6  AAEXK
476      1673   6  19  25  28  36   1  MFCHM
477      1672   1   2  25  31  33  14  AVFBK
478      1671  15  28  36  37  41  10  MHADP

[477 rows x 8 columns]
Printing rejected
     idSorteo  n1  n2  n3  n4  n5  sb  delta
451      1698   1  27  31  34  36  11  YDCBH
467      1682  12  13  25  31  42  16  ALFKM


In [250]:

df1 = df_re_fu_filt
choosen, rejected = validator2_m(df1, dft)
print("Printing choosen ")
print(choosen)
print("Printing rejected")
print(rejected)

Printing choosen 
     idSorteo  n1  n2  n3  n4  n5  sb  delta
0        2149  14  15  26  28  40  14  AKBLP
1        2148   5  20  35  38  43   2  ÑÑCEE
2        2147   4   7  19  33  39   8  CLNFH
3        2146   7  25  34  42  43   2  QIHAG
4        2145   9  16  19  29  39   1  GCJJM
..        ...  ..  ..  ..  ..  ..  ..    ...
474      1675   3   5   8  20  24  14  BCLDU
475      1674   5  12  15  20  42  14  GCEUF
476      1673   8  21  22  37  38   4  MAÑAM
477      1672   9  12  16  28  38   1  CDLJN
478      1671   3   9  10  21  40  10  FAKRF

[479 rows x 8 columns]
Printing rejected
Empty DataFrame
Columns: [idSorteo, n1, n2, n3, n4, n5, sb, delta]
Index: []


In [251]:
quint, cuart, trios = couples_builder(dft)
df_reduced = choosen
cuart = couples_builder_sorteos(df_reduced)




#choosen, rejected = validator(quint, cuart, trios, candidates, rejected)
print(quint)

2002
5
10010
20020
5
[30, 31, 34, 36, 42]


In [252]:
# def validator_t(trios):
#     trios_df = pd.DataFrame (cuart, columns = ['n1','n2','n3','n4'])
#     validar_sumar_resultados_sorteos_t(n1,n2,n3,trios_df)
#     elegidos_t, rechazados, totaltrios  = sumar_tuplas_t(n1,n2,n3,df_excluidos)
#     if (totaltrios <= 4):
#         choosen.append(row)
#         totales = (totalquintetos/ 2) + (totalcuartetos/10) + (totaltrios/20)
#         print("total quintetos: " + str(totalquintetos) + ", total cuartetos: " + str(totalcuartetos) + ", total trios: " + str(totaltrios))
#         print("totales: "+ str(totales))
#         print("---------")
#     else: 
#         rejected.append(row)

In [253]:
# def validator_c(cuart, trios):
#     cuart_df = pd.DataFrame (cuart, columns = ['n1','n2','n3','n4'])
#     can_elegidos = len(dft)
#     for index, row in cuart.iterrows():
#         sbs = (row)
#         n1 = int(sbs[0])
#         n2 = int(sbs[1])
#         n3 = int(sbs[2])
#         n4 = int(sbs[3])
#         porcentaje = ((i+1)/can_elegidos)*100
#         validar_sumar_resultados_sorteos_c(n1,n2,n3,n4, cuart_df)
#         choose_c, rejected_c, totalcuartetos  = sumar_tuplas_c(n1,n2,n3,n4, df_excluidos) #, df_cc_tr, df_cc_re)
#         if (totalcuartetos == 0):
#             validator_t(trios)
#         else:
#             rejected.append(row)
#         return 

In [254]:
# def validator(quint, cuart, trios, dft ):
#     i = 0
#     dictionary = {}
#     can_elegidos = len(dft)
#     #can_elegidos = 1000
#     print(can_elegidos)
#     validos=[]
#     choosen = []
#     rejected = []
#     rejecteds = []
#     df_excluidos = []
#     df_excluidos = df_tr_fu_filt
#     iteracion = 0
#     for index, row in dft.iterrows():
#         sbs = (row)
#         # print(type(sbs[0]))
#         n1 = int(sbs[0])
#         n2 = int(sbs[1])
#         n3 = int(sbs[2])
#         n4 = int(sbs[3])
#         n5 = int(sbs[4]) 
#         porcentaje = ((i+1)/can_elegidos)*100
#         print("Iteracion " + str(i) + " Porcentaje Avance: "+ str(porcentaje), " N1:" + str(n1), " N2:" + str(n2), " N3:" + str(n3), " N4:" + str(n4), " N5:" + str(n5))
#         totalquintetos = 0
#         choosen, rejected, totalquintetos  = sumar_tuplas(n1,n2,n3,n4,n5, df_excluidos)
#         iteracion = iteracion + 1
#         if (totalquintetos == 0):
#             print("longitud cuartetos: " + str(len(cuart)))
#             #validator_c(cuart)
#         else:
#             print("total Quintetos != 0")
#             rejecteds.append(rejected)
#         if (iteracion >= can_elegidos):
#             salir = int(input("ingresa si quieres seguir evaluando total quintetos > 0: "))
#             if(salir > 0 ):
#                 return choosen, rejected
#         totales = 0
#         totaltrios = 0
#         i = i + 1
#     return choosen, rejected


In [255]:
# df1 = pd.DataFrame({'c': ['A', 'A', 'B', 'C', 'C'],
#                     'k': [1, 2, 2, 2, 2],
#                     'l': ['a', 'b', 'a', 'a', 'd']})
# df2 = pd.DataFrame({'c': ['A', 'C'],
#                     'l': ['b', 'a']})
# print(df1)
# print(df2)


In [256]:
# keys = list(df2.columns.values)
# print(keys)
# i1 = df1.set_index(keys).index
# i2 = df2.set_index(keys).index
# dfinal = df1[~i1.isin(i2)]
# dfinal2 = df1[i1.isin(i2)]
# print (df1)

In [257]:
# dfinal

In [258]:
# dfinal2

In [259]:
choosen = []
rejected = []
#choosen, rejected = validator(quint, cuart, trios, dft)

In [260]:
len(choosen)

0

In [261]:

len(rejected)

0

In [262]:
# import ipykernel  
# ipykernel.__version__


In [263]:
# import sys
# sys.version